In [17]:
import torch
import torch.nn as nn

batch_size = 4


class SampleConcatener(nn.Module):
    def __init__(self, input_X_dim, input_t_dim, input_s_dim, output_dim, hidden=32):
        super(SampleConcatener, self).__init__()
        self.mlp_embedder = nn.Linear(input_X_dim, hidden)
        self.temporal_embedder = nn.Linear(input_t_dim, lat*lon)
        self.mlp_decoder = nn.Linear(hidden + 1 + input_s_dim, output_dim)

    def forward(self, X, t, s):
        X = self.mlp_embedder(X).relu()
        print(X.shape)
        # (batch, s, temp_features) -> (batch, 1, s*temp_features) -> (batch, 1, lat*lon) -> (batch, lat*lon, 1)
        print(t.shape)
        t = self.temporal_embedder(t.reshape(batch_size, 1, -1)).relu().permute((0,2,1))
        print(t.shape)
        print(s.shape)
        concat = torch.cat((X, t, s), dim=-1)
        print(concat.shape)
        
        print('----------')
        print('GNN cells ...')                
        output = self.mlp_decoder(concat)
        print('----------')
        print(output.shape)
        return output

lat = 25
lon = 45    
s = 3
features = 6
temp_features = 3
spat_features = 3

X = torch.randn(batch_size, lat*lon, s*features)
temp_enc = torch.randn(batch_size, s, temp_features)
spat_enc = torch.randn(batch_size, lat*lon, spat_features)

model = SampleConcatener(input_X_dim=s*features, input_t_dim=s*temp_features, input_s_dim=spat_features, output_dim=features)

output = model(X, temp_enc, spat_enc)


torch.Size([4, 1125, 32])
torch.Size([4, 3, 3])
torch.Size([4, 1125, 1])
torch.Size([4, 1125, 3])
torch.Size([4, 1125, 36])
----------
GNN cells ...
----------
torch.Size([4, 1125, 6])
